## Processing temperature files

In [1]:
%%capture
!pip install luigi pyyaml netCDF4 h5netcdf scipy

In [2]:
%%capture
!pip install boto3 s3fs xarray numpy zarr pyyaml tqdm

In [3]:
%%capture
!pip install "xarray[complete]"==2023.8.0 s3fs --user

In [4]:
import xarray as xr
import pandas as pd
import numpy as np
import s3fs
import os
import re
from collections import defaultdict
import zarr
#import blosc
from numcodecs import blosc
from tqdm import tqdm
from glob import glob

In [5]:
def kelvin_a_celsius(temp_k):
    return xr.where(np.isnan(temp_k), temp_k, temp_k - 273.15)

In [6]:
def extraer_fecha_y_tipo(nombre_archivo):
    match = re.search(r'Temperature-Air-2m-(Max|Mean|Min)-24h.*_(\d{8})_', nombre_archivo)
    if match:
        tipo, fecha = match.groups()
        return fecha, tipo
    return '00000000', ''

In [7]:
def ordenar_archivos(directorio):
    archivos = [f for f in os.listdir(directorio) if f.endswith('.nc')]
    archivos_por_fecha = defaultdict(dict)
    for archivo in archivos:
        fecha, tipo = extraer_fecha_y_tipo(archivo)
        archivos_por_fecha[fecha][tipo] = archivo
    fechas_ordenadas = sorted(archivos_por_fecha.keys())
    archivos_ordenados = []
    for fecha in fechas_ordenadas:
        for tipo in ['Max', 'Mean', 'Min']:
            if tipo in archivos_por_fecha[fecha]:
                archivos_ordenados.append(archivos_por_fecha[fecha][tipo])
    return archivos_ordenados

In [8]:
def procesar_archivos_mes(año, mes, ruta_local_entrada, bucket_salida, ruta_salida_s3):
    fecha_inicio = pd.Timestamp(f"{año}-{mes:02d}-01")
    fecha_fin = fecha_inicio + pd.offsets.MonthEnd(0)
    
    archivos_ordenados = ordenar_archivos(ruta_local_entrada)
    archivos_mes = [archivo for archivo in archivos_ordenados 
                    if fecha_inicio <= pd.to_datetime(extraer_fecha_y_tipo(archivo)[0]) <= fecha_fin]
    
    datasets = []
    
    for i in range(0, len(archivos_mes), 3):  # Procesar en grupos de 3 (Max, Mean, Min)
        ds_dia = []
        for j in range(3):
            if i+j < len(archivos_mes):
                archivo = archivos_mes[i+j]
                ds = xr.open_dataset(os.path.join(ruta_local_entrada, archivo))
                fecha, tipo = extraer_fecha_y_tipo(archivo)
                var_name = f'Temperature_Air_2m_{tipo}_24h'
                ds[var_name] = kelvin_a_celsius(ds[var_name])
                ds[var_name].attrs['units'] = 'C'
                ds_dia.append(ds)
        
        if ds_dia:
            ds_combinado = xr.merge(ds_dia)
            datasets.append(ds_combinado)
    
    if datasets:
        ds_final = xr.concat(datasets, dim='time')
        
        # Configurar el sistema de archivos S3
        s3 = s3fs.S3FileSystem(anon=False)
        zarr_store = s3fs.S3Map(root=f's3://{bucket_salida}/{ruta_salida_s3}', s3=s3)
        
        # Configurar compresión y encoding
        compressor = zarr.Blosc(cname='lz4', clevel=1, shuffle=False)
        blosc.set_nthreads(8)
        encoding = {vname: {'compressor': compressor, 'chunks': (1, 1, 2000, 7200)} for vname in ds_final.data_vars}
        
        # Verificar si el archivo Zarr ya existe
        try:
            existing_ds = xr.open_zarr(zarr_store)
            mode = 'a'
            # Remover el encoding para variables existentes
            for vname in existing_ds.data_vars:
                if vname in encoding:
                    del encoding[vname]
            append_dim = 'time'
        except:
            mode = 'w'
            append_dim = None
        
        # Escribir a Zarr en S3
        with tqdm(total=100, desc=f"Procesando {año}-{mes:02d}") as pbar:
            ds_final.to_zarr(zarr_store, mode=mode, append_dim=append_dim, encoding=encoding, consolidated=True)
            pbar.update(100)
        
        print(f"Datos procesados y guardados en S3: s3://{bucket_salida}/{ruta_salida_s3}")
    else:
        print(f"No se encontraron datos para el mes {mes} del año {año}")

In [9]:
# Ejecución cuando el zarr no existe
ruta_local_entrada = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/tmp_test'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=testTemperatureAir.zarr'

# Procesar un mes específico
procesar_archivos_mes(1980, 1, ruta_local_entrada, bucket_salida, ruta_salida_s3)

Procesando 1980-01: 100%|██████████| 100/100 [02:09<00:00,  1.29s/it]

Datos procesados y guardados en S3: s3://climate-action-datalake/zone=raw/source=agera5-v1-1/variable=testTemperatureAir.zarr


In [11]:
# Configurar conexión a S3
s3 = s3fs.S3FileSystem(anon=False)
# Ubicación del archivo Zarr en S3
bucket_name = 'climate-action-datalake'
zarr_path = 'zone=raw/source=agera5-v1-1/variable=testTemperatureAir.zarr'
s3_url_1 = f's3://{bucket_name}/{zarr_path}'
# Abrir el archivo Zarr
ds = xr.open_zarr(s3fs.S3Map(s3_url_1, s3=s3))
ds

<xarray.Dataset> Size: 2GB
Dimensions:                      (time: 31, lat: 1801, lon: 3600)
Coordinates:
  * lat                          (lat) float64 14kB 90.0 89.9 ... -89.9 -90.0
  * lon                          (lon) float64 29kB -180.0 -179.9 ... 179.9
  * time                         (time) datetime64[ns] 248B 1980-01-01 ... 19...
Data variables:
    Temperature_Air_2m_Max_24h   (time, lat, lon) float32 804MB dask.array<chunksize=(4, 226, 450), meta=np.ndarray>
    Temperature_Air_2m_Mean_24h  (time, lat, lon) float32 804MB dask.array<chunksize=(4, 226, 450), meta=np.ndarray>
    Temperature_Air_2m_Min_24h   (time, lat, lon) float32 804MB dask.array<chunksize=(4, 226, 450), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.2 (http://mpimet.mpg.de/...
    CDO:          Climate Data Operators version 1.9.2 (http://mpimet.mpg.de/...
    Conventions:  CF-1.7
    history:      Fri Mar 12 13:28:08 2021: cdo splitday /archive/ESG/wit015/...

In [12]:
# Ejecución con zarr ya existe
ruta_local_entrada = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/test'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=testTemperatureAir.zarr'

# Procesar un mes específico
procesar_archivos_mes(2024, 1, ruta_local_entrada, bucket_salida, ruta_salida_s3)

Procesando 2024-01: 100%|██████████| 100/100 [02:30<00:00,  1.50s/it]

Datos procesados y guardados en S3: s3://climate-action-datalake/zone=raw/source=agera5-v1-1/variable=testTemperatureAir.zarr


In [13]:
# Abrir el archivo Zarr
ds2 = xr.open_zarr(s3fs.S3Map(s3_url_1, s3=s3))
ds2

<xarray.Dataset> Size: 5GB
Dimensions:                      (time: 62, lat: 1801, lon: 3600)
Coordinates:
  * lat                          (lat) float64 14kB 90.0 89.9 ... -89.9 -90.0
  * lon                          (lon) float64 29kB -180.0 -179.9 ... 179.9
  * time                         (time) datetime64[ns] 496B 1980-01-01 ... 20...
Data variables:
    Temperature_Air_2m_Max_24h   (time, lat, lon) float32 2GB dask.array<chunksize=(4, 226, 450), meta=np.ndarray>
    Temperature_Air_2m_Mean_24h  (time, lat, lon) float32 2GB dask.array<chunksize=(4, 226, 450), meta=np.ndarray>
    Temperature_Air_2m_Min_24h   (time, lat, lon) float32 2GB dask.array<chunksize=(4, 226, 450), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7

In [17]:
print(f"Zarr: {ds2.dims}")

Zarr: FrozenMappingWarningOnValuesAccess({'time': 62, 'lat': 1801, 'lon': 3600})


In [ ]:
# Arturo
    #compressor = zarr.Blosc(cname='lz4', clevel= 1, shuffle=False)
    #blosc.set_nthreads(8) 
    #encoding = {vname: {'compressor': compressor,'chunks': (1,1,2000,7200)} for vname in nc.data_vars}
    #with ProgressBar():
        #nc.to_zarr(ageraS3,  mode='a', append_dim='time', consolidated=True)

## Desglosando el proceso paso a paso

In [ ]:
ruta_local_entrada = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/tmp_test'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=testTemperature_Air_2m_24h.zarr'

In [ ]:
mes = 1
año = 1980
fecha_inicio = pd.Timestamp(f"{año}-{mes:02d}-01")
fecha_fin = fecha_inicio + pd.offsets.MonthEnd(0)
print(fecha_inicio)
print(fecha_fin)

In [ ]:
# Probar funcion procesar_archivos_mes
archivos_ordenados = ordenar_archivos(ruta_local_entrada)
archivos_mes = [archivo for archivo in archivos_ordenados
                if fecha_inicio <= pd.to_datetime(extraer_fecha_y_tipo(archivo)[0]) <= fecha_fin]

In [ ]:
#pip install --upgrade s3fs

In [ ]:
#pip install "s3fs>=2023.1.0"

In [ ]:
#pip install --upgrade botocore awscli boto3

In [ ]:
#pip show botocore awscli boto3

In [ ]:
# Verificar si el archivo Zarr ya existe y obtener las fechas existentes
try:
    existing_ds = xr.open_zarr(s3_url_1)
    existing_times = existing_ds.time.values
    mode = 'a'
except:
    existing_times = []
    mode = 'w'

In [ ]:
existing_times

In [ ]:
# Configurar la conexión a S3
s3 = s3fs.S3FileSystem(anon=False)

# Especificar la ubicación del archivo Zarr en S3
bucket_name = 'climate-action-datalake'
zarr_path1 = 'zone=raw/source=agera5-v1-1/variable=testTemperature_Air_2m_24h.zarr'
zarr_path2 = 'zone=raw/source=agera5-v1-1/variable=Temperature_Air_2m_24h.zarr'
s3_url_1 = f's3://{bucket_name}/{zarr_path1}'
s3_url_2 = f's3://{bucket_name}/{zarr_path2}'

In [ ]:
# Abrir el archivo Zarr
ds1 = xr.open_zarr(s3fs.S3Map(s3_url_1, s3=s3))
ds2 = xr.open_zarr(s3fs.S3Map(s3_url_2, s3=s3))

In [ ]:
print(ds1)

In [ ]:
print(ds2)

In [ ]:
ds1

## Prueba 3

In [ ]:
#pip install dask[complete] zarr xarray[io]

In [ ]:
%%capture
!pip install --upgrade s3fs

In [ ]:
%%capture
!pip install "xarray[complete]"==2023.8.0 s3fs --user

In [21]:
import os
import re
import xarray as xr
import zarr
import pandas as pd
import numpy as np
from collections import defaultdict
from dask.diagnostics import ProgressBar

In [29]:
nombre_archivo = 'Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_20240101_final-v1.1.nc'

In [30]:
def extraer_fecha_y_tipo(nombre_archivo):
    match = re.search(r'Temperature-Air-2m-(Max|Mean|Min)-24h.*_(\d{8})_', nombre_archivo)
    if match:
        tipo, fecha = match.groups()
        return fecha, tipo
    return '00000000', ''

def ordenar_archivos(directorio):
    archivos = [f for f in os.listdir(directorio) if f.endswith('.nc')]
    archivos_por_fecha = defaultdict(dict)
    for archivo in archivos:
        fecha, tipo = extraer_fecha_y_tipo(archivo)
        archivos_por_fecha[fecha][tipo] = archivo
    fechas_ordenadas = sorted(archivos_por_fecha.keys())
    archivos_ordenados = []
    for fecha in fechas_ordenadas:
        for tipo in ['Max', 'Mean', 'Min']:
            if tipo in archivos_por_fecha[fecha]:
                archivos_ordenados.append(archivos_por_fecha[fecha][tipo])
    return archivos_ordenados

def kelvin_a_celsius(temp_k):
    return xr.where(np.isnan(temp_k), temp_k, temp_k - 273.15)

In [31]:
extraer_fecha_y_tipo(nombre_archivo)

('20240101', 'Max')

In [23]:
archivos_ordenados = ordenar_archivos('/home/ec2-user/SageMaker/datalake/data/agera/temperature/test')
archivos_ordenados[0]

'Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_20240101_final-v1.1.nc'

In [33]:
def ordenar_archivos(directorio):
    archivos = [f for f in os.listdir(directorio) if f.endswith('.nc')]
    archivos_por_fecha = defaultdict(dict)
    for archivo in archivos:
        fecha, tipo = extraer_fecha_y_tipo(archivo)
        archivos_por_fecha[fecha][tipo] = archivo
    fechas_ordenadas = sorted(archivos_por_fecha.keys())
    archivos_ordenados = []
    for fecha in fechas_ordenadas:
        for tipo in ['Max', 'Mean', 'Min']:
            if tipo in archivos_por_fecha[fecha]:
                archivos_ordenados.append(archivos_por_fecha[fecha][tipo])
    return archivos_ordenados

In [34]:
archivos_ordenados = ordenar_archivos('/home/ec2-user/SageMaker/datalake/data/agera/temperature/test')
archivos_ordenados[0]

'Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_20240101_final-v1.1.nc'

In [27]:
def procesar_archivos_mes(año, mes, ruta_local_entrada, bucket_salida, ruta_salida_s3):
    fecha_inicio = pd.Timestamp(f"{año}-{mes:02d}-01")
    fecha_fin = fecha_inicio + pd.offsets.MonthEnd(0)
    
    archivos_ordenados = ordenar_archivos(ruta_local_entrada)
    archivos_mes = [archivo for archivo in archivos_ordenados 
                    if fecha_inicio <= pd.to_datetime(extraer_fecha_y_tipo(archivo)[0]) <= fecha_fin]
    
    paths = [os.path.join(ruta_local_entrada, archivo) for archivo in archivos_mes]
    
    ds = xr.open_mfdataset(paths, chunks='auto')
    
    # Aplicar la conversión de Kelvin a Celsius
    for var in ds.data_vars:
        ds[var] = kelvin_a_celsius(ds[var])
    
    compressor = zarr.Blosc(cname='lz4', clevel= 1, shuffle=False)
    encoding = {var: {'compressor': compressor, 'chunks': ds[var].data.chunksize} for var in ds.data_vars}
    
    s3_path = f"s3://{bucket_salida}/{ruta_salida_s3}"
    with ProgressBar():
        result = ds.to_zarr(s3_path, mode='a', append_dim='time', encoding=encoding, consolidated=True, compute=False)
        result.compute(scheduler='processes')

In [28]:
ruta_local_entrada = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/tmp_test'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=Temperature_Air_2m_24h.zarr'
año, mes = 1980, 1
procesar_archivos_mes(año, mes, ruta_local_entrada, bucket_salida, ruta_salida_s3)

[########################################] | 100% Completed | 10.31 s


In [25]:
ruta_local_entrada = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/tmp_test'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=Temperature_Air_2m_24h.zarr'
año, mes = 1980, 1

In [29]:
fecha_inicio = pd.Timestamp(f"{año}-{mes:02d}-01")
fecha_fin = fecha_inicio + pd.offsets.MonthEnd(0)

In [30]:
archivos_ordenados = ordenar_archivos(ruta_local_entrada)
archivos_mes = [archivo for archivo in archivos_ordenados 
                if fecha_inicio <= pd.to_datetime(extraer_fecha_y_tipo(archivo)[0]) <= fecha_fin]

In [31]:
paths = [os.path.join(ruta_local_entrada, archivo) for archivo in archivos_mes]

In [32]:
len(paths)

93

In [33]:
paths[0]

'/home/ec2-user/SageMaker/datalake/data/agera/temperature/tmp_test/Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_19800101_final-v1.1.nc'

In [34]:
ds = xr.open_mfdataset(paths)

In [35]:
for var in ds.data_vars:
        ds[var] = kelvin_a_celsius(ds[var])

In [36]:
compressor = zarr.Blosc(cname='lz4')
encoding = {var: {'compressor': compressor, 'chunks': (1, 901, 1800)} for var in ds.data_vars}

In [ ]:
s3_path = f"s3://{bucket_salida}/{ruta_salida_s3}"
with ProgressBar():
    result = ds.to_zarr(s3_path, encoding=encoding, consolidated=True, compute=False)
    result.compute(scheduler='processes')

In [37]:
# Intenta abrir el grupo Zarr existente
try:
    existing_ds = xr.open_zarr(s3_path)
    append_dim = 'time'  # Asume que 'time' es tu dimensión temporal
    
    # Encuentra el último tiempo en el dataset existente
    last_time = existing_ds.time.max().values
    
    # Filtra el nuevo dataset para incluir solo tiempos posteriores al último tiempo existente
    ds = ds.sel(time=ds.time > last_time)
    
    if len(ds.time) > 0:
        with ProgressBar():
            result = ds.to_zarr(s3_path, append_dim=append_dim, encoding=encoding, consolidated=True, compute=False)
            result.compute(scheduler='processes')
        print(f"Datos agregados exitosamente a {s3_path}")
    else:
        print("No hay nuevos datos para agregar.")

except zarr.errors.GroupNotFoundError:
    # Si el grupo no existe, crea uno nuevo
    with ProgressBar():
        result = ds.to_zarr(s3_path, encoding=encoding, consolidated=True, compute=False)
        result.compute(scheduler='processes')
    print(f"Nuevo archivo Zarr creado en {s3_path}")

No hay nuevos datos para agregar.


In [26]:
import s3fs

# Configurar la conexión a S3
s3 = s3fs.S3FileSystem(anon=False)

# Especificar la ubicación del archivo Zarr en S3
bucket_name = 'climate-action-datalake'
zarr_path = 'zone=raw/source=agera5-v1-1/variable=Temperature_Air_2m_24h.zarr'
s3_url = f's3://{bucket_name}/{zarr_path}'

# Abrir el archivo Zarr
ds = xr.open_zarr(s3fs.S3Map(s3_url, s3=s3))

# Mostrar información básica sobre el dataset
print(ds)

<xarray.Dataset>
Dimensions:                      (time: 31, lat: 1801, lon: 3600)
Coordinates:
  * lat                          (lat) float64 90.0 89.9 89.8 ... -89.9 -90.0
  * lon                          (lon) float64 -180.0 -179.9 ... 179.8 179.9
  * time                         (time) datetime64[ns] 2024-01-01 ... 2024-01-31
Data variables:
    Temperature_Air_2m_Max_24h   (time, lat, lon) float32 dask.array<chunksize=(1, 901, 1800), meta=np.ndarray>
    Temperature_Air_2m_Mean_24h  (time, lat, lon) float32 dask.array<chunksize=(1, 901, 1800), meta=np.ndarray>
    Temperature_Air_2m_Min_24h   (time, lat, lon) float32 dask.array<chunksize=(1, 901, 1800), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7


In [27]:
ds

<xarray.Dataset>
Dimensions:                      (time: 31, lat: 1801, lon: 3600)
Coordinates:
  * lat                          (lat) float64 90.0 89.9 89.8 ... -89.9 -90.0
  * lon                          (lon) float64 -180.0 -179.9 ... 179.8 179.9
  * time                         (time) datetime64[ns] 2024-01-01 ... 2024-01-31
Data variables:
    Temperature_Air_2m_Max_24h   (time, lat, lon) float32 dask.array<chunksize=(1, 901, 1800), meta=np.ndarray>
    Temperature_Air_2m_Mean_24h  (time, lat, lon) float32 dask.array<chunksize=(1, 901, 1800), meta=np.ndarray>
    Temperature_Air_2m_Min_24h   (time, lat, lon) float32 dask.array<chunksize=(1, 901, 1800), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7

In [39]:
# Especificar la ubicación del archivo Zarr en S3
bucket_name = 'climate-action-datalake'
zarr_path = 'zone=raw/source=agera5-v1-1/variable=testTemperature_Air_2m_24h.zarr'
s3_url = f's3://{bucket_name}/{zarr_path}'

# Abrir el archivo Zarr
ds = xr.open_zarr(s3fs.S3Map(s3_url, s3=s3))
ds

<xarray.Dataset>
Dimensions:                      (time: 31, lat: 1801, lon: 3600)
Coordinates:
  * lat                          (lat) float64 90.0 89.9 89.8 ... -89.9 -90.0
  * lon                          (lon) float64 -180.0 -179.9 ... 179.8 179.9
  * time                         (time) datetime64[ns] 1980-01-01 ... 1980-01-31
Data variables:
    Temperature_Air_2m_Max_24h   (time, lat, lon) float32 dask.array<chunksize=(4, 226, 450), meta=np.ndarray>
    Temperature_Air_2m_Mean_24h  (time, lat, lon) float32 dask.array<chunksize=(4, 226, 450), meta=np.ndarray>
    Temperature_Air_2m_Min_24h   (time, lat, lon) float32 dask.array<chunksize=(4, 226, 450), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.2 (http://mpimet.mpg.de/...
    CDO:          Climate Data Operators version 1.9.2 (http://mpimet.mpg.de/...
    Conventions:  CF-1.7
    history:      Fri Mar 12 13:28:08 2021: cdo splitday /archive/ESG/wit015/...

In [ ]:
ruta_local_entrada = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/test'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=Temperature_Air_2m_24h.zarr'
año, mes = 2024, 1

In [ ]:
# Uso
ruta_local_entrada = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/tmp_test'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=testTemperature_Air.zarr'
año, mes = 1980, 1
procesar_archivos_mes(año, mes, ruta_local_entrada, bucket_salida, ruta_salida_s3)

## Comparar una carpeta de archivos .nc con un archivo zarr en s3

In [14]:
def validar_zarr_vs_nc(ruta_local_nc, bucket_salida, ruta_zarr_s3):
    # Configurar acceso a S3
    s3 = s3fs.S3FileSystem(anon=False)
    ruta_completa_s3 = f's3://{bucket_salida}/{ruta_zarr_s3}'

    # Abrir el archivo Zarr desde S3
    ds_zarr = xr.open_zarr(s3fs.S3Map(ruta_completa_s3, s3=s3))

    # Patrones para los archivos de cada tipo de temperatura
    patrones = {
        'min': '*Min*.nc',
        'mean': '*Mean*.nc',
        'max': '*Max*.nc'
    }

    datasets_nc = []

    # Abrir y combinar todos los archivos NC
    for tipo, patron in patrones.items():
        archivos = glob(os.path.join(ruta_local_nc, patron))
        ds = xr.open_mfdataset(archivos, combine='by_coords')
        var_name = f'Temperature_Air_2m_{tipo.capitalize()}_24h'
        
        # Convertir de Kelvin a Celsius para comparar con Zarr
        ds[var_name] = ds[var_name] - 273.15
        ds[var_name].attrs['units'] = 'C'
        
        datasets_nc.append(ds)

    ds_nc = xr.merge(datasets_nc)

    # Comparar las formas de los datasets
    print("Comparación de formas:")
    print(f"Zarr: {ds_zarr.dims}")
    print(f"NC: {ds_nc.dims}")
    if ds_zarr.dims == ds_nc.dims:
        print("Las dimensiones coinciden.")
    else:
        print("ADVERTENCIA: Las dimensiones no coinciden.")

    # Comparar variables
    print("\nComparación de variables:")
    for var in ds_zarr.data_vars:
        print(f"\nVariable: {var}")
        
        # Comparar estadísticas básicas
        zarr_stats = ds_zarr[var].compute()
        nc_stats = ds_nc[var].compute()
        
        print(f"  Media Zarr: {zarr_stats.mean().values:.4f}")
        print(f"  Media NC: {nc_stats.mean().values:.4f}")
        print(f"  Desviación Estándar Zarr: {zarr_stats.std().values:.4f}")
        print(f"  Desviación Estándar NC: {nc_stats.std().values:.4f}")
        print(f"  Mínimo Zarr: {zarr_stats.min().values:.4f}")
        print(f"  Mínimo NC: {nc_stats.min().values:.4f}")
        print(f"  Máximo Zarr: {zarr_stats.max().values:.4f}")
        print(f"  Máximo NC: {nc_stats.max().values:.4f}")

        # Calcular y mostrar la diferencia máxima absoluta
        diff = np.abs(zarr_stats - nc_stats)
        max_diff = diff.max().values
        print(f"  Diferencia máxima absoluta: {max_diff:.6f}")
        
        # Calcular y mostrar el error cuadrático medio (RMSE)
        rmse = np.sqrt(((zarr_stats - nc_stats) ** 2).mean()).values
        print(f"  Error cuadrático medio (RMSE): {rmse:.6f}")

        # Calcular y mostrar el coeficiente de correlación
        corr = xr.corr(zarr_stats, nc_stats).values
        print(f"  Coeficiente de correlación: {corr:.6f}")

        # Comparar número de NaN
        nan_zarr = np.isnan(zarr_stats).sum().values
        nan_nc = np.isnan(nc_stats).sum().values
        print(f"  Número de NaN en Zarr: {nan_zarr}")
        print(f"  Número de NaN en NC: {nan_nc}")
        if nan_zarr == nan_nc:
            print("  El número de NaN coincide.")
        else:
            print(f"  ADVERTENCIA: El número de NaN no coincide. Diferencia: {abs(nan_zarr - nan_nc)}")

    print("\nValidación completada.")

In [ ]:
ruta_local_nc = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/tmp_test'
bucket_salida = 'climate-action-datalake'
ruta_zarr_s3 = 'zone=raw/source=agera5-v1-1/variable=testTemperature_Air_2m_24h.zarr'
validar_zarr_vs_nc(ruta_local_nc, bucket_salida, ruta_zarr_s3)

## Fin